In [2]:
import pandas as pd
import re
from datetime import datetime

In [3]:
# warn_nasdaq_df = pd.read_csv(
#     "../data/warn_jaccard_0.5.csv",
#     delimiter=","
# )
# warn_nasdaq_df = warn_nasdaq_df.rename({"id2": "Symbol"}, axis=1)
# warn_nasdaq_df = warn_nasdaq_df.drop_duplicates(subset=["OG-Company-Warn", "Symbol"])
# warn_nasdaq_df = warn_nasdaq_df.sort_values("jaccard", ascending=True)
# warn_nasdaq_df.to_csv("../warn_jaccard_0.5_dedup.csv", index=False)

In [4]:
warn_nasdaq_cleaned_df = pd.read_csv("../data/warn_jaccard_0.5_cleaned.csv")
warn_nasdaq_cleaned_df = warn_nasdaq_cleaned_df.drop(["id1", "joinKey1", "joinKey2", "jaccard"], axis=1)
warn_nasdaq_cleaned_df = warn_nasdaq_cleaned_df.drop_duplicates(subset=["OG-Company-Warn", "Symbol"])
warn_nasdaq_cleaned_df = warn_nasdaq_cleaned_df.set_index("OG-Company-Warn")
print(warn_nasdaq_cleaned_df['Symbol'].unique().shape)
warn_nasdaq_cleaned_df.head()

(579,)


,Symbol,OG-Company-Nasdaq
OG-Company-Warn,,
Credit Suisse Asset Management LLC (CSAM),CIK,Credit Suisse Asset Management Income Fund Inc...
"G-III Apparel Group, Ltd. (3 NYC sites)",GIII,G-III Apparel Group LTD. Common Stock
Royal Caribbean Cruises Ltd.,RCL,D/B/A Royal Caribbean Cruises Ltd. Common Stock
M&T Bank (Mid-Hudson),MTB,M&T Bank Corporation Common Stock
"Delta Air Lines, Inc-Pilot Base",DAL,Delta Air Lines Inc. Common Stock


In [5]:
warn_layoffs_df = pd.read_csv("../data/warn_1.csv", index_col='id')
warn_layoffs_df["Date"] = warn_layoffs_df["WARN Received Date"].fillna(warn_layoffs_df["Effective Date"])
warn_layoffs_df["Number of Workers"] = warn_layoffs_df["Number of Workers"].str.strip()
warn_layoffs_df["Number of Workers"] = warn_layoffs_df["Number of Workers"].str.replace(r",","", regex=False)
warn_layoffs_df = warn_layoffs_df[~warn_layoffs_df["Number of Workers"].isna()]
warn_layoffs_df = warn_layoffs_df[warn_layoffs_df["Number of Workers"] != '']
warn_layoffs_df = warn_layoffs_df.astype({'Date': 'datetime64[ns]', 'Number of Workers': 'int64'})
warn_layoffs_df = warn_layoffs_df[[
    "State", "City", "Company", "Number of Workers", "Date"
]]
warn_layoffs_df = warn_layoffs_df.drop_duplicates()
# print(warn_layoffs_df.shape)
# # Limit analysis to 2010 and after
warn_layoffs_df = warn_layoffs_df[warn_layoffs_df["Date"] > pd.to_datetime("2010-01-01")]
print(warn_layoffs_df.shape)
warn_layoffs_df.head()

(38686, 5)


/var/folders/z2/drm596w97t97nxjngc05sxk00000gn/T/ipykernel_91568/3386901416.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  warn_layoffs_df = pd.read_csv("../data/warn_1.csv", index_col='id')


,State,City,Company,Number of Workers,Date
id,,,,,
0,Alabama,Daleville,"Primus Solutions, LLC",132,2021-07-28
1,Alabama,Jasper,Corsicana Mattress Company,42,2021-07-26
2,Alabama,Birmingham,Portfolio Recovery Associates LLC,133,2021-07-20
3,Alabama,Gadsden,Crothall Healthcare Environmental Services,58,2021-07-07
4,Alabama,Decatur,National Packaging Co. Inc.,62,2021-06-30


In [6]:
layoffs_symbol_df = warn_layoffs_df.join(other=warn_nasdaq_cleaned_df, how="inner", on="Company")
layoffs_symbol_df = layoffs_symbol_df.reset_index(drop=True)
layoffs_symbol_df = layoffs_symbol_df.drop(["Company"], axis=1)
layoffs_symbol_df = layoffs_symbol_df.rename({"OG-Company-Nasdaq": "Company"}, axis=1)
layoffs_symbol_df = layoffs_symbol_df[~layoffs_symbol_df["Number of Workers"].isna()]
layoffs_symbol_df = layoffs_symbol_df[["Date", "Symbol", "Company", "Number of Workers"]]
layoffs_symbol_df = layoffs_symbol_df.groupby(by=["Date", "Symbol", "Company"], as_index=False).sum()
layoffs_symbol_df = layoffs_symbol_df.sort_values(by="Date", ascending=True).reset_index(drop=True)
print(f"Layoff events count = {layoffs_symbol_df.shape[0]}")
print(f"Unique layoff company count = {layoffs_symbol_df['Symbol'].unique().shape[0]}")
layoffs_symbol_df

Layoff events count = 2110
Unique layoff company count = 527


,Date,Symbol,Company,Number of Workers
0,2010-01-05,M,Macy's Inc Common Stock,72
1,2010-01-06,MHK,Mohawk Industries Inc. Common Stock,190
2,2010-01-07,AXP,American Express Company Common Stock,55
3,2010-01-14,BAC,Bank of America Corporation Common Stock,50
4,2010-01-14,IP,International Paper Company Common Stock,2
...,...,...,...,...
2105,2023-01-10,JBL,Jabil Inc. Common Stock,205
2106,2023-01-11,PSX,Phillips 66 Common Stock,45
2107,2023-01-12,ANF,Abercrombie & Fitch Company Common Stock,11
2108,2023-01-18,INTC,Intel Corporation Common Stock,544


In [14]:
fmp_df = pd.read_csv("../data/combined_company_financials.csv")
fmp_df = fmp_df.rename({'date': 'statement_date'}, axis=1).astype({'statement_date': 'datetime64[ns]'})
fmp_df = fmp_df.set_index("symbol", drop=True).sort_index(na_position="first")
# print(fmp_df.columns.to_list())
fmp_df = fmp_df[['statement_date', 'revenue', 'netIncome', 'EPS']]
fmp_df = fmp_df[fmp_df['statement_date'] >= pd.to_datetime("2008-01-01")]
print(fmp_df.shape)
fmp_df.head()

(31442, 4)


,statement_date,revenue,netIncome,EPS
symbol,,,,
A,2008-04-30,1.456000e+09,173000000.0,0.48
A,2008-07-31,1.444000e+09,169000000.0,0.47
A,2008-10-31,1.481000e+09,231000000.0,0.66
A,2009-01-31,1.166000e+09,64000000.0,0.18
A,2009-04-30,1.091000e+09,-101000000.0,-0.29


In [38]:
o_fmp_df = fmp_df.reset_index().sort_values(['symbol', 'statement_date'], ascending=True).reset_index(drop=True)
o_fmp_df

,symbol,statement_date,revenue,netIncome,EPS
0,A,2008-01-31,1.393000e+09,120000000.0,0.32
1,A,2008-04-30,1.456000e+09,173000000.0,0.48
2,A,2008-07-31,1.444000e+09,169000000.0,0.47
3,A,2008-10-31,1.481000e+09,231000000.0,0.66
4,A,2009-01-31,1.166000e+09,64000000.0,0.18
...,...,...,...,...,...
31437,ZUMZ,2022-01-29,3.466770e+08,38204000.0,1.73
31438,ZUMZ,2022-04-30,2.206860e+08,-397000.0,-0.02
31439,ZUMZ,2022-07-30,2.199930e+08,3067000.0,0.16
31440,ZUMZ,2022-10-29,2.375910e+08,6932000.0,0.36


In [42]:
def add_fmp_prev_period(o_fmp_df, n_fmp_df, n_periods_ago=1):
    suffix = f"_t-{n_periods_ago}"
    n_fmp_df = pd.concat([n_fmp_df, o_fmp_df.shift(n_periods_ago).add_suffix(suffix)], axis=1)
    n_fmp_df = n_fmp_df[n_fmp_df['symbol']==n_fmp_df[f'symbol{suffix}']]
    n_fmp_df = n_fmp_df.drop([f'symbol{suffix}'], axis=1)
    return n_fmp_df

In [47]:
# n_fmp_df = pd.concat([o_fmp_df, o_fmp_df.shift(1).add_suffix('_t-1')], axis=1)
# n_fmp_df = n_fmp_df[n_fmp_df['symbol']==n_fmp_df['symbol_t-1']]
# dropped_columns = filter(lambda c: c.startswith('prev') and c != 'prev_statement_date', n_fmp_df.columns.tolist())
# n_fmp_df = n_fmp_df.drop(list(dropped_columns), axis=1)
# n_fmp_df = n_fmp_df.set_index('symbol')
# n_fmp_df
n_fmp_df = add_fmp_prev_period(o_fmp_df, o_fmp_df, 1)
n_fmp_df = add_fmp_prev_period(o_fmp_df, n_fmp_df, 2)
n_fmp_df = add_fmp_prev_period(o_fmp_df, n_fmp_df, 3)
n_fmp_df = add_fmp_prev_period(o_fmp_df, n_fmp_df, 4)
n_fmp_df = n_fmp_df.set_index('symbol')
n_fmp_df

,statement_date,revenue,netIncome,EPS,statement_date_t-1,revenue_t-1,netIncome_t-1,EPS_t-1,statement_date_t-2,revenue_t-2,netIncome_t-2,EPS_t-2,statement_date_t-3,revenue_t-3,netIncome_t-3,EPS_t-3,statement_date_t-4,revenue_t-4,netIncome_t-4,EPS_t-4
symbol,,,,,,,,,,,,,,,,,,,,
A,2009-01-31,1.166000e+09,64000000.0,0.180,2008-10-31,1.481000e+09,231000000.0,0.660,2008-07-31,1.444000e+09,169000000.0,0.47,2008-04-30,1.456000e+09,173000000.0,0.48,2008-01-31,1.393000e+09,120000000.0,0.32
A,2009-04-30,1.091000e+09,-101000000.0,-0.290,2009-01-31,1.166000e+09,64000000.0,0.180,2008-10-31,1.481000e+09,231000000.0,0.66,2008-07-31,1.444000e+09,169000000.0,0.47,2008-04-30,1.456000e+09,173000000.0,0.48
A,2009-07-31,1.057000e+09,-19000000.0,-0.060,2009-04-30,1.091000e+09,-101000000.0,-0.290,2009-01-31,1.166000e+09,64000000.0,0.18,2008-10-31,1.481000e+09,231000000.0,0.66,2008-07-31,1.444000e+09,169000000.0,0.47
A,2009-10-31,1.167000e+09,25000000.0,0.072,2009-07-31,1.057000e+09,-19000000.0,-0.060,2009-04-30,1.091000e+09,-101000000.0,-0.29,2009-01-31,1.166000e+09,64000000.0,0.18,2008-10-31,1.481000e+09,231000000.0,0.66
A,2010-01-31,1.213000e+09,79000000.0,0.230,2009-10-31,1.167000e+09,25000000.0,0.072,2009-07-31,1.057000e+09,-19000000.0,-0.06,2009-04-30,1.091000e+09,-101000000.0,-0.29,2009-01-31,1.166000e+09,64000000.0,0.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZUMZ,2022-01-29,3.466770e+08,38204000.0,1.730,2021-10-30,2.894550e+08,30702000.0,1.260,2021-07-31,2.686660e+08,23998000.0,0.95,2021-05-01,2.790690e+08,26385000.0,1.05,2021-01-30,3.315360e+08,42797000.0,1.71
ZUMZ,2022-04-30,2.206860e+08,-397000.0,-0.020,2022-01-29,3.466770e+08,38204000.0,1.730,2021-10-30,2.894550e+08,30702000.0,1.26,2021-07-31,2.686660e+08,23998000.0,0.95,2021-05-01,2.790690e+08,26385000.0,1.05
ZUMZ,2022-07-30,2.199930e+08,3067000.0,0.160,2022-04-30,2.206860e+08,-397000.0,-0.020,2022-01-29,3.466770e+08,38204000.0,1.73,2021-10-30,2.894550e+08,30702000.0,1.26,2021-07-31,2.686660e+08,23998000.0,0.95


In [55]:
warn_fmp_df = layoffs_symbol_df.join(other=n_fmp_df, on="Symbol", how="inner")
warn_fmp_df['has_layoffs'] = warn_fmp_df.apply(
    lambda r: 1 if (r['Date'] > r['statement_date_t-1'] and r['Date'] <= r['statement_date']) else 0,
    axis=1)
warn_fmp_df = warn_fmp_df.sort_values(['Symbol', 'Date', 'statement_date'], ascending=True)
warn_fmp_df = warn_fmp_df.reset_index(drop=True)
warn_fmp_df

,Date,Symbol,Company,Number of Workers,statement_date,revenue,netIncome,EPS,statement_date_t-1,revenue_t-1,...,EPS_t-2,statement_date_t-3,revenue_t-3,netIncome_t-3,EPS_t-3,statement_date_t-4,revenue_t-4,netIncome_t-4,EPS_t-4,has_layoffs
0,2015-11-09,AA,Alcoa Corporation Common Stock,415,2016-03-31,2.129000e+09,-203000000.0,-1.112939,2015-12-31,2.451000e+09,...,-0.679825,2015-06-30,2.964000e+09,-97000000.0,-0.531800,2015-03-31,3.105000e+09,166000000.0,0.910088,0
1,2015-11-09,AA,Alcoa Corporation Common Stock,415,2016-06-30,2.323000e+09,-55000000.0,-0.301535,2016-03-31,2.129000e+09,...,-4.528510,2015-09-30,2.679000e+09,-124000000.0,-0.679825,2015-06-30,2.964000e+09,-97000000.0,-0.531800,0
2,2015-11-09,AA,Alcoa Corporation Common Stock,415,2016-09-30,2.329000e+09,-6000000.0,-0.032883,2016-06-30,2.323000e+09,...,-1.112939,2015-12-31,2.451000e+09,-826000000.0,-4.528510,2015-09-30,2.679000e+09,-124000000.0,-0.679825,0
3,2015-11-09,AA,Alcoa Corporation Common Stock,415,2016-12-31,2.537000e+09,-139000000.0,-0.680000,2016-09-30,2.329000e+09,...,-0.301535,2016-03-31,2.129000e+09,-203000000.0,-1.112939,2015-12-31,2.451000e+09,-826000000.0,-4.528510,0
4,2015-11-09,AA,Alcoa Corporation Common Stock,415,2017-03-31,2.655000e+09,225000000.0,1.230000,2016-12-31,2.537000e+09,...,-0.032883,2016-06-30,2.323000e+09,-55000000.0,-0.301535,2016-03-31,2.129000e+09,-203000000.0,-1.112939,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111458,2015-12-28,ZUMZ,Zumiez Inc. Common Stock,61,2022-01-29,3.466770e+08,38204000.0,1.730000,2021-10-30,2.894550e+08,...,0.950000,2021-05-01,2.790690e+08,26385000.0,1.050000,2021-01-30,3.315360e+08,42797000.0,1.710000,0
111459,2015-12-28,ZUMZ,Zumiez Inc. Common Stock,61,2022-04-30,2.206860e+08,-397000.0,-0.020000,2022-01-29,3.466770e+08,...,1.260000,2021-07-31,2.686660e+08,23998000.0,0.950000,2021-05-01,2.790690e+08,26385000.0,1.050000,0
111460,2015-12-28,ZUMZ,Zumiez Inc. Common Stock,61,2022-07-30,2.199930e+08,3067000.0,0.160000,2022-04-30,2.206860e+08,...,1.730000,2021-10-30,2.894550e+08,30702000.0,1.260000,2021-07-31,2.686660e+08,23998000.0,0.950000,0
111461,2015-12-28,ZUMZ,Zumiez Inc. Common Stock,61,2022-10-29,2.375910e+08,6932000.0,0.360000,2022-07-30,2.199930e+08,...,-0.020000,2022-01-29,3.466770e+08,38204000.0,1.730000,2021-10-30,2.894550e+08,30702000.0,1.260000,0


In [99]:
warn_fmp_df = layoffs_symbol_df.join(other=fmp_df, on="Symbol", how="inner")
warn_fmp_df = warn_fmp_df[warn_fmp_df['statement_date'] < warn_fmp_df['Date']]
warn_fmp_df = warn_fmp_df[(warn_fmp_df['Date'] - warn_fmp_df['statement_date']) <= pd.Timedelta("365 days")]
warn_fmp_df = warn_fmp_df.drop_duplicates(subset=['Symbol', 'Company', 'Date', 'Number of Workers', 'statement_date'])
warn_fmp_df = warn_fmp_df.sort_values(['Date', 'Symbol', 'statement_date']).reset_index(drop=True)
print(warn_fmp_df.shape)
warn_fmp_df.tail(20)

(8190, 8)


,Date,Symbol,Company,Number of Workers,statement_date,revenue,netIncome,EPS
8170,2023-01-10,JBL,Jabil Inc. Common Stock,205,2022-02-28,7.553000e+09,2.220000e+08,1.55
8171,2023-01-10,JBL,Jabil Inc. Common Stock,205,2022-05-31,8.328000e+09,2.180000e+08,1.55
8172,2023-01-10,JBL,Jabil Inc. Common Stock,205,2022-08-31,9.030000e+09,3.150000e+08,2.30
8173,2023-01-10,JBL,Jabil Inc. Common Stock,205,2022-11-30,9.635000e+09,2.230000e+08,1.65
8174,2023-01-11,PSX,Phillips 66 Common Stock,45,2022-03-31,3.617900e+10,5.820000e+08,1.32
8175,2023-01-11,PSX,Phillips 66 Common Stock,45,2022-06-30,4.857700e+10,3.167000e+09,6.55
8176,2023-01-11,PSX,Phillips 66 Common Stock,45,2022-09-30,4.495500e+10,5.391000e+09,11.19
8177,2023-01-11,PSX,Phillips 66 Common Stock,45,2022-12-31,4.599100e+10,1.884000e+09,4.00
8178,2023-01-12,ANF,Abercrombie & Fitch Company Common Stock,11,2022-01-29,1.161353e+09,6.550900e+07,1.18
8179,2023-01-12,ANF,Abercrombie & Fitch Company Common Stock,11,2022-04-30,8.127620e+08,-1.646900e+07,-0.32


In [98]:
new_df = None
grouped = warn_fmp_df.groupby(by=['Date', 'Symbol', 'Company', 'Number of Workers'])
for name, group in grouped:
    if group['statement_date'].shape[0] == 4:
        group['statement_date'] = ['t-1','t-2','t-3','t-4']
        if new_df is None:
            new_df = group
        else:
            new_df = pd.concat([new_df, group], ignore_index=True)
    # print(group.shape)
# warn_fmp_df
warn_fmp_df = new_df
warn_fmp_df

,Date,Symbol,Company,Number of Workers,statement_date,revenue,netIncome,EPS
0,2010-01-05,M,Macy's Inc Common Stock,72,t-1,7.934000e+09,-4.773000e+09,-11.329219
1,2010-01-05,M,Macy's Inc Common Stock,72,t-2,5.199000e+09,-8.800000e+07,-0.210000
2,2010-01-05,M,Macy's Inc Common Stock,72,t-3,5.164000e+09,7.000000e+06,0.020000
3,2010-01-05,M,Macy's Inc Common Stock,72,t-4,5.277000e+09,-3.500000e+07,-0.080000
4,2010-01-06,MHK,Mohawk Industries Inc. Common Stock,190,t-1,1.208339e+09,-1.058870e+08,-1.550000
...,...,...,...,...,...,...,...,...
8051,2023-01-18,INTC,Intel Corporation Common Stock,544,t-4,1.404200e+10,-6.640000e+08,-0.160000
8052,2023-01-18,MSFT,Microsoft Corporation Common Stock,878,t-1,4.936000e+10,1.672800e+10,2.230000
8053,2023-01-18,MSFT,Microsoft Corporation Common Stock,878,t-2,5.186500e+10,1.674000e+10,2.240000
8054,2023-01-18,MSFT,Microsoft Corporation Common Stock,878,t-3,5.012200e+10,1.755600e+10,2.350000


In [9]:
# pivoted_df = warn_fmp_df[['State', 'City', 'Symbol', 'Company', 'Date', 'statement_date', 'revenue', 'costOfRevenue']]
pivoted_df = warn_fmp_df.drop(["Number of Workers"], axis=1)
pivoted_df = pivoted_df.set_index(['State', 'City', 'Symbol', 'Company', 'Date', 'statement_date'])
# pivoted_df.apply()
# pivoted_df = warn_fmp_df.set_index(['State', 'City', 'Symbol', 'Company', 'Date', 'Number of Workers'])
pivoted_df = pivoted_df.unstack().reset_index()
pivoted_df.columns = pivoted_df.columns.map(lambda c: ":".join(filter(len, c)))
# pivoted_df.columns = pivoted_df.columns.to_flat_index()
pivoted_df

,State,City,Symbol,Company,Date,revenue:t-1,revenue:t-2,revenue:t-3,revenue:t-4,netIncome:t-1,netIncome:t-2,netIncome:t-3,netIncome:t-4,EPS:t-1,EPS:t-2,EPS:t-3,EPS:t-4
0,Alabama,Anniston,SAIC,SCIENCE APPLICATIONS INTERNATIONAL CORPORATION...,2012-06-11,2.596000e+09,2.811000e+09,2.492000e+09,2.782000e+09,178000000.0,-91000000.0,-1.610000e+08,117000000.0,0.541033,-0.270000,-0.489362,0.355623
1,Alabama,Athens,OC,Owens Corning Inc Common Stock New,2002-01-30,1.165000e+09,1.291000e+09,1.239000e+09,1.067000e+09,-7000000.0,27000000.0,2.900000e+07,-10000000.0,-0.127042,0.500000,0.530000,-0.190000
2,Alabama,Athens,PPC,Pilgrim's Pride Corporation Common Stock,2009-07-24,2.169489e+09,1.876991e+09,1.698102e+09,1.776813e+09,-802926000.0,-228782000.0,-5.876500e+07,53239000.0,-10.842190,-3.090000,-0.790000,0.720000
3,Alabama,Bessemer,CAH,Cardinal Health Inc. Common Stock,2008-04-24,2.290960e+10,2.226290e+10,2.328270e+10,2.197340e+10,356000000.0,902200000.0,3.247000e+08,301800000.0,1.000000,2.485399,0.910000,0.830000
4,Alabama,Birmingham,CAH,Cardinal Health Inc. Common Stock,2006-08-08,2.063750e+10,2.170830e+10,1.991900e+10,1.937340e+10,146800000.0,321000000.0,3.040000e+08,228300000.0,0.350000,0.793572,0.710000,0.540000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2723,Wisconsin,Racine,XRAY,DENTSPLY SIRONA Inc. Common Stock,2019-03-27,1.059700e+09,9.284000e+08,1.042100e+09,9.561000e+08,1800000.0,28000000.0,-1.122000e+09,81200000.0,0.010000,0.130000,-4.980000,0.360000
2724,Wisconsin,Ripon,SJM,J.M. Smucker Company (The) New Common Stock,2022-03-08,1.920200e+09,2.057100e+09,2.050000e+09,1.858000e+09,147000000.0,69700000.0,2.060000e+08,153900000.0,1.350000,0.640000,1.900000,1.420000
2725,Wisconsin,Tomah,SND,Smart Sand Inc. Common Stock,2020-04-28,4.766700e+07,6.569000e+07,6.794100e+07,4.748800e+07,2388000.0,10926000.0,1.427600e+07,-84000.0,0.060000,0.270000,0.360000,-0.002095
2726,Wisconsin,Wausau,DLX,Deluxe Corporation Common Stock,2021-04-01,4.104050e+08,4.394610e+08,4.544930e+08,4.412640e+08,14859000.0,29417000.0,2.466300e+07,24292000.0,0.360000,0.701788,0.588474,0.580000
